# Training

## Setup

In [ ]:
import os
import string
# Name of the new model
Custom_Model_Name = 'sign_language_detection_model_v44'
Pretrained_Model_Name = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
# Name of TF Record Script
TF_Record_Script_Name = 'generate_tfrecord.py'
# Name of label map
Label_Map_Name = 'label_map.pbtxt'

In [ ]:
# Creates the file hierarchy for the project
paths = {
    'Annotation': os.path.join('Tensorflow','Data','Annotations'),
    'Scripts': os.path.join('Tensorflow', 'Scripts'),
    'Images': os.path.join('Tensorflow', 'Data', 'Images'),
    'Pretrained_Model': os.path.join('Tensorflow', 'Pretrained_Models'),
    'Checkpoint': os.path.join('Tensorflow', 'Models', Custom_Model_Name),
    'APIModel_Path': os.path.join('Tensorflow','Models_API'),
}

for path in paths.values():
    if not os.path.exists(path):
        !mkdir {path}

In [ ]:
files = {
    'LabelMap': os.path.join(paths['Annotation'], Label_Map_Name),
    'TF_Record_Script': os.path.join(paths['Scripts'], TF_Record_Script_Name),
    'Pipeline_Config': os.path.join('Tensorflow', 'Models', Custom_Model_Name, 'pipeline.config'),
}

## 1. Creating the Label Map

In [ ]:
# Array containing the name of each label
labels = []
labels.append({'name': 'Accept', 'id': 1})
labels.append({'name': 'Delete', 'id': 2})
i = 3
for label in string.ascii_uppercase:
    if label != 'J' and label != 'Z':
        labels.append({'name': label, 'id': i})
        i += 1

# Writes the Label Map 
with open(files['LabelMap'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write(f'\tname:\'{label["name"]}\'\n')
        f.write(f'\tid:{label["id"]}\n')
        f.write('}\n')

## 2. Generating TFRecords

In [ ]:
# Clones the script for generating TFRecords
if not os.path.exists(files['TF_Record_Script']):
    !git clone https://github.com/nicknochnack/GenerateTFRecord {paths['Scripts']}

In [ ]:
# Creates TFRecords for both training and testing
!python {files['TF_Record_Script']} -x {os.path.join(paths['Images'], 'train')} -l {files['LabelMap']} -o {os.path.join(paths['Annotation'], 'train.record')}
!python {files['TF_Record_Script']} -x {os.path.join(paths['Images'], 'test')} -l {files['LabelMap']} -o {os.path.join(paths['Annotation'], 'test.record')}

## 3. Update Pipeline Configuration

In [ ]:
# Copy's the pipeline configuration of the pretrained model
!copy {os.path.join(paths['Pretrained_Model'], Pretrained_Model_Name, 'pipeline.config')} {os.path.join(paths['Checkpoint'])}

In [ ]:
# Imports TensorFlow
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [ ]:
# Returns dictionary of configuration objects
config = config_util.get_configs_from_pipeline_file(files['Pipeline_Config'])

In [ ]:
# Create pipeline object
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
# Merge the pipeline of pretrained model to the object
with tf.io.gfile.GFile(files['Pipeline_Config'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config) 

In [ ]:
# Change number of total classes
pipeline_config.model.ssd.num_classes = len(labels)
# Change batch size
pipeline_config.train_config.batch_size = 4
# Set the checkpoint path to the pretrained model checkpoint
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['Pretrained_Model'], Pretrained_Model_Name, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
# Set the paths to the new Label Maps and TFRecords
pipeline_config.train_input_reader.label_map_path= files['LabelMap']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['Annotation'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LabelMap']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['Annotation'], 'test.record')]

In [ ]:
# Writes the updated pipeline config
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['Pipeline_Config'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

## 4. Training

In [ ]:
# Path to the object detection API script for training and evaluation
Training_Script = os.path.join(paths['APIModel_Path'], 'research', 'object_detection', 'model_main_tf2.py')

In [ ]:
training_command = f'python {Training_Script} --model_dir={paths["Checkpoint"]} --pipeline_config_path={files["Pipeline_Config"]} --num_train_steps=2000'

In [ ]:
# The command is run manually in the terminal so the progress can be seen
print(training_command)

## 5. Freeze the Graph

In [ ]:
# Path to the object detection API script for exporting
FREEZE_SCRIPT = os.path.join(paths['APIModel_Path'], 'research', 'object_detection', 'exporter_main_v2.py ')

In [ ]:
command = "python {} --input_type=image_tensor --pipeline_config_path={} --trained_checkpoint_dir={} --output_directory={}".format(FREEZE_SCRIPT ,files['Pipeline_Config'], paths['Checkpoint'], paths['Output'])

In [ ]:
!{command}

## 6. Convert to TensorFlow.js

In [ ]:
# Install TensorFlow.js
!pip install tensorflowjs

In [ ]:
command = "tensorflowjs_converter --input_format=tf_saved_model --output_node_names='detection_boxes,detection_classes,detection_features,detection_multiclass_scores,detection_scores,num_detections,raw_detection_boxes,raw_detection_scores' --output_format=tfjs_graph_model --signature_name=serving_default {} {}".format(os.path.join(paths['Output'], 'saved_model'), paths['TFJS'])

In [ ]:
!{command}

## 7. Evaluation

In [ ]:
evaluation_command = f'python {Training_Script} --model_dir={paths["Checkpoint"]} --pipeline_config_path={files["Pipeline_Config"]} --checkpoint_dir={paths["Checkpoint"]}'

In [ ]:
# The command is run manually in the terminal so the progress can be seen
print(evaluation_command)

## Citation

@misc{tensorflowmodelgarden2020, author = {Hongkun Yu, Chen Chen, Xianzhi Du, Yeqing Li, Abdullah Rashwan, Le Hou, Pengchong Jin, Fan Yang, Frederick Liu, Jaeyoun Kim, and Jing Li}, title = {{TensorFlow Model Garden}}, howpublished = {\url{https://github.com/tensorflow/models}}, year = {2020} }